# Artificial Intelligence
## L.EIC – 3rd Year/2nd Semester
### Exercise Sheet 1
# Solving Problems by Searching

## 1.1 The Two Buckets Problem

<img src="https://qph.cf2.quoracdn.net/main-qimg-45726b16b460cae0147ae8ca245a8fb0-pjlq" width="250px" height="250px" align="right">

Two buckets of capacities **c1** (e.g. 4 liters) and **c2** (e.g. 3 liters), respectively, are initially empty. The buckets do not have any intermediate markings. The only operations you can perform are:

- Fill (completely) a bucket 
- Empty a bucket.
- Pour one bucket into the other (until the target one is full or the source one is empty).

The aim is to determine which operations to carry out so that the first bucket contains exactly **n** liters (e.g. 2 litres).

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

What is the size of the state space for this problem? Represent the state space by drawing the possible objective states and displaying some of the possible transitions from the initial state.

<div align="center">
    <img src="state_space.jpg">
</div>

Solve the problem by hand, using tree search. What solutions have you found?

### Buildind a computational approach to handle the problem

To build a program to solve the buckets problem, we will implement a solution that separates the problem definition from the algorithms used to traverse the state space. This way, we can reuse our implementations of the search strategies in other problems.

#### Representing the two buckets problem as a search problem

Let's start by defining a state for the buckets problem. For that, it'll suffice to aggregate two quantities, each representing the amount of water in one of the buckets. We also define a way of printing the state.

In [4]:
class BucketState:
    c1 = 4   # capacity for bucket 1
    c2 = 3   # capacity for bucket 2
    
    def __init__(self, b1, b2):
        self.b1 = b1
        self.b2 = b2

    '''needed for the visited list'''
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    def __hash__(self):
        return hash((self.b1, self.b2)) 
    ''' - '''

    def __str__(self):
        return "(" + str(self.b1) + ", " + str(self.b2) + ")"

Now we define each of the operators on states:

In [5]:
# emptying the first bucket
def empty1(state):
    if state.b1 > 0:
        return BucketState(0, state.b2)
    return None

# emptying the second bucket
def empty2(state):
    if state.b2 > 0:
        return BucketState(state.b1, 0)
    return None
    
# fill the first bucket
def fill1(state):
    if state.b1 < 4:
        return BucketState(4, state.b2)
    return None

# fill the second bucket
def fill2(state):
    if state.b2 < 3:
        return BucketState(state.b1, 3)
    return None

# pour water from bucket one into bucket two until bucket two is full
def pour12_fill2(state):
    if state.b1 > 0 and state.b2 < 3 and state.b1+state.b2 >= 3:
        return BucketState(state.b1-(3-state.b2), 3)
    return None

# pour water from bucket one into bucket two until bucket one is empty
def pour12_empty1(state):
    if state.b1 > 0 and state.b2 < 3 and state.b1+state.b2 < 3:
        return BucketState(0, state.b1+state.b2)
    return None

# pour water from bucket two into bucket one until bucket one is full
def pour21_fill1(state):
    if state.b2 > 0 and state.b1 < 4 and state.b1+state.b2 >= 4:
        return BucketState(4, state.b2-(4-state.b1))
    return None

# pour water from bucket two into bucket one until bucket two is empty
def pour21_empty2(state):
    if state.b2 > 0 and state.b1 < 4 and state.b1+state.b2 < 4:
        return BucketState(state.b1+state.b2, 0)
    return None


The following function will aggregate all states that can be generated from a given one:

In [6]:
def child_bucket_states(state):
    new_states = []
    if(empty1(state)):
        new_states.append(empty1(state))
    if(empty2(state)):
        new_states.append(empty2(state))
    if(fill1(state)):
        new_states.append(fill1(state))
    if(fill2(state)):
        new_states.append(fill2(state))
    if(pour12_fill2(state)):
        new_states.append(pour12_fill2(state))
    if(pour12_empty1(state)):
        new_states.append(pour12_empty1(state))
    if(pour21_fill1(state)):
        new_states.append(pour21_fill1(state))
    if(pour21_empty2(state)):
        new_states.append(pour21_empty2(state))
    return new_states

Play around with the state transition operators and check if they are working properly:

In [7]:
s = BucketState(0, 0)
# print(s)  # (0, 0)
s = fill1(s)
print(s)  # (4, 0)

c = child_bucket_states(BucketState(0, 0))

for i in child_bucket_states(s):
  print(i)

#s = pour12_fill2(s)
#print(s)  # (1, 3)

#s = pour21_fill1(s)
#print(s) # (4, 0)

#s = fill2(s)
#print(s)  # (4, 3)

#s = empty1(s)
#print(s) # (0,3)

#s = empty2(s)
#print(s)  # (0, 0)

(4, 0)
(0, 0)
(4, 3)
(1, 3)


Finally, we need to define the goal condition:

In [8]:
def goal_bucket_state(state):
   return state.b1 == 2

Test your goal condition:

In [9]:
print(goal_bucket_state(BucketState(0, 0))) # False
print(goal_bucket_state(BucketState(4, 3))) # False
print(goal_bucket_state(BucketState(2, 0))) # True

False
False
True


#### Implementing search algorithms

Let us start by defining an appropriate structure to represent a node in a search tree. Each tree node will include:
- a state of the problem
- a link to its parent (to allow traveling from a leaf node towards the root of the tree)
- a list of child nodes

In [10]:
# A generic definition of a tree node holding a state of the problem
class TreeNode:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        if parent is None:
            self.depth = 0
        else: 
            self.depth = self.parent.depth + 1

    def add_child(self, child_node):
        self.children.append(child_node)
        child_node.parent = self
        child_node.depth = self.depth + 1

##### Breadth-first search

Based on this structure, we can now implement breadth-first search. Note that we want the implementation to be independent of the problem at hand (in this case, the two buckets problem).

In [11]:
from collections import deque

def breadth_first_search(initial_state, goal_state_func, operators_func):
    root = TreeNode(initial_state)   # create the root node in the search tree
    queue = deque([root])   # initialize the queue to store the nodes
    
    while queue:
        node = queue.popleft()   # get first element in the queue
        if goal_state_func(node.state):   # check goal state
            return node
        
        for state in operators_func(node.state):   # go through next states
            # create tree node with the new state
            new_node = TreeNode(state) 

            # link child node to its parent in the tree
            node.add_child(new_node)
            
            # enqueue the child node
            queue.append(new_node)

    return None

We can now use this function to actually perform a breadth-first search on the buckets problem: we pass it the initial state, our goal condition function, and the function for obtaining child states.

In [12]:
goal = breadth_first_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states)
print(goal.state)   # (2, 3)

(2, 3)


In order to print the actual steps from the initial state to the last, we can take advantage of each node's link to its parent.

In [13]:
def print_solution(node):
    steps = []

    while node.parent is not None:
        steps.append(node.state)
        node = node.parent

    steps.append(BucketState(0,0))
    steps.reverse()
    
    print(f"Found goal state in {len(steps)-1} steps:")
    
    for step in steps:
        print(step)

    return

Now we can print the solution:

In [14]:
print_solution(goal)

# Found goal state in 6 steps:
# (0, 0)
# (4, 0)
# (1, 3)
# (1, 0)
# (0, 1)
# (4, 1)
# (2, 3)

Found goal state in 6 steps:
(0, 0)
(4, 0)
(1, 3)
(1, 0)
(0, 1)
(4, 1)
(2, 3)


If we need a description for each of the employed operators, we could have each operation function return also such a description, and modify the TreeNode class so that each node also includes a description of the edge to get there. We leave that as an exercise after class.

##### Depth-first search

Implement depth-first search (again, in a manner that is independent of the problem at hand). You can start from your breadth-first search implementation and with minor changes get an implementation for depth-first search.

In [15]:
from collections import deque

def depth_first_search(initial_state, goal_state_func, operators_func):
        root = TreeNode(initial_state)   # create the root node in the search tree
        stack = deque([root])   # initialize the stack to store the nodes
        visited = {initial_state}  # visited states to avoid infite loop

        while stack:
            node = stack.pop()   # get last element in the stack

            if goal_state_func(node.state):   # check goal state
                return node

            for state in operators_func(node.state):   # go through next states
                if state not in visited:  # check if state has not been visited before
                    # create tree node with the new state
                    new_node = TreeNode(state)

                    # link child node to its parent in the tree
                    node.add_child(new_node)

                    # add the child node to the stack
                    stack.append(new_node)

                    # add the child node to the visited set
                    visited.add(state)

        return None

Test it on the two buckets problem.

In [16]:
goal = depth_first_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states)

print(goal.state)
print_solution(goal)

# (2, 0)
# Found goal state in 6 steps:
# (0, 0)
# (0, 3)
# (3, 0)
# (3, 3)
# (4, 2)
# (0, 2)
# (2, 0)

(2, 0)
Found goal state in 6 steps:
(0, 0)
(0, 3)
(3, 0)
(3, 3)
(4, 2)
(0, 2)
(2, 0)


If you are unable to get a solution, think about it: depth-first search is not a complete search method, and one of the reasons for that is if the state space contains cycles. As such, you need to make sure you avoid entering into a cycle by keeping a visited nodes list or set and checking that list whenever you generate a new state.

##### Depth-limited search

Another way to make it work is to impose a depth limit to the problem. Implement depth-limited search.

In [29]:
def depth_limited_search(initial_state, goal_state_func, operators_func, depth_limit):
    visited = set([initial_state])  # set of visited states to avoid infite loops

    def sub_dfs(node, depth):
        if goal_state_func(node.state):
            return node
        
        if depth == depth_limit:
            return None
        
        visited.add(node.state)

        for state in operators_func(node.state):
            new_node = TreeNode(state = state, parent = node)
            result = sub_dfs(new_node, depth + 1)
            if result is not None:
                return result
        return None
            
    return sub_dfs(TreeNode(initial_state), 0)

Test it on the two buckets problem.

In [30]:
goal = depth_limited_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states, 
                            6)

print(goal.state)
print_solution(goal)

# (2, 3)
# Found goal state in 6 steps:
# (0, 0)
# (4, 0)
# (1, 3)
# (1, 0)
# (0, 1)
# (4, 1)
# (2, 3)

(2, 3)
Found goal state in 6 steps:
(0, 0)
(4, 0)
(1, 3)
(1, 0)
(0, 1)
(4, 1)
(2, 3)


##### Iterative deepening search

Based on depth-limited, you can easily implement iterative-deepening search.

In [31]:
def iterative_deepening_search(initial_state, goal_state_func, operators_func, depth_limit):
    for depth in range(depth_limit):
        result = depth_limited_search(initial_state, goal_state_func, operators_func, depth)
        if result is not None:
            return result
    return None

Again, test it on the two buckets problem.

In [32]:
goal = iterative_deepening_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states, 
                            7)

print(goal.state)
print_solution(goal)

# (2, 3)
# Found goal state in 6 steps:
# (0, 0)
# (4, 0)
# (1, 3)
# (1, 0)
# (0, 1)
# (4, 1)
# (2, 3)

(2, 3)
Found goal state in 6 steps:
(0, 0)
(4, 0)
(1, 3)
(1, 0)
(0, 1)
(4, 1)
(2, 3)


##### Greedy Search
Implement code to solve two buckets problem using Greedy Search.

In [33]:
def heuristic_bucket(node):
    return abs(node.state.b1-2)

In [34]:

def greedy_search(initial_state, goal_state_func, operators_func, heuristic_func):
    root = TreeNode(initial_state)  # create the root node in the search tree
    queue = [(root, heuristic_func(root))]  # initialize the queue to store the nodes
    visited = set([initial_state])

    while queue:
        queue.sort(key=lambda node: node[1])  # sort the queue based on heuristic cost
        (node, _) = queue.pop(0)  # get the node with the lowest heuristic cost

        if goal_state_func(node.state):  # check goal state
            return node

        for state in operators_func(node.state):  # go through next states
            if state not in visited:  # check if state has not been visited before
                # create tree node with the new state
                new_node = TreeNode(state)

                # link child node to its parent in the tree
                node.add_child(new_node)

                # add the child node to the queue with its heuristic cost
                queue.append((new_node, heuristic_func(new_node)))

                # add the child node to the visited set
                visited.add(state)

    return None

Test it in the two buckets problem.

In [35]:
goal = greedy_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states,
                            heuristic_bucket)

print(goal.state)
print_solution(goal)

(2, 3)
Found goal state in 6 steps:
(0, 0)
(4, 0)
(1, 3)
(1, 0)
(0, 1)
(4, 1)
(2, 3)


##### A* Algorithm

In [61]:
from collections import deque

def a_star_search(initial_state, goal_state_func, operators_func, heuristic_func):
    root = TreeNode(initial_state)   # create the root node in the search tree
    queue = deque([root])   # initialize the queue to store the nodes

    while queue:
        node = min(queue, key=lambda n: heuristic_func(n) + n.depth)    # selects the node with the lowest cost from the queue

        if goal_state_func(node.state):   # check goal state
            return node

        queue.remove(node)  # remove node from the queue so that it is not explored again

        for state in operators_func(node.state):   # go through next states
            # create tree node with the new state
            new_node = TreeNode(state=state, parent=node)

            # link child node to its parent in the tree
            node.add_child(new_node)

            # enqueue the child node
            queue.append(new_node)

    return None

Test it in the two buckets problem.

In [62]:
goal = a_star_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states,
                            heuristic_bucket)

print(goal.state)
print_solution(goal)

(2, 3)
Found goal state in 6 steps:
(0, 0)
(4, 0)
(1, 3)
(1, 0)
(0, 1)
(4, 1)
(2, 3)


## 1.2 The Missionaries and Cannibals Problem 

<img src="https://allfish24.files.wordpress.com/2016/09/gambar-1.jpg">

Three missionaries and three cannibals are on one of the banks of the river with a boat that only takes one or two people. The boat cannot travel the river alone.

The goal is to find a way to get the six to the other bank of the river without ever leaving more cannibals than missionaries on one of the banks (even at the instant they leave/join the boat) during the process.

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

Solve the problem by hand, using tree search. What solutions have you found?

Represent the problem as a search problem and take advantage of the implemented search algorithms to find solutions!

In [1]:
class RiverState:
    def __init__(self, m, c, b):
        self.m = m
        self.c = c
        self.b = b

    def __eq__(self, other):
        return self.m == other.m and self.c == other.c and self.b == other.b
    
    def __ne__(self, other):
        return not self.__eq__(other)
    
    def __hash__(self):
        return hash((self.m, self.c, self.b))
    
    def __str__(self):
        return "Side 1: (" + str(self.m) + ", " + str(self.c) + ", " + str(self.b) + ")" + "Side 2: + (" + str(3-self.m) + ", " + str(3-self.c) + ", " + str(3-self.b) + ")"

In [2]:
# Take one missionary from side 1 to 2
def m1_12(state):
     if state.m > state.c and state.m > 0 and state.b == 1:
            return RiverState(state.m-1, state.c, 0)
     return None

# Take one missionary from side 2 to 1
def m1_21(state):
     if 3-state.m > 3-state.c and 3-state.m > 0 and state.b == 0:
            return RiverState(state.m+1, state.c, 1)
     return None

# Take two missionaries from side 1 to 2
def m2_12(state):
     if state.m > state.c+1 and state.m > 1 and state.b == 1:
            return RiverState(state.m-2, state.c, 0)
     return None

# Take two missionaries from side 2 to 1
def m2_21(state):
     if 3-state.m > 3-state.c+1 and 3-state.m > 1 and state.b == 0:
            return RiverState(state.m+2, state.c, 1)
     return None

# Take one cannibal from side 1 to 2
def c1_12(state):
     if state.c > 0 and state.b == 1:
            return RiverState(state.m, state.c-1, 0)
     return None

# Take one cannibal from side 2 to 1
def c1_21(state):
     if 3-state.c > 0 and state.b == 0:
            return RiverState(state.m, state.c+1, 1)
     return None

# Take two cannibals from side 1 to 2
def c2_12(state):
     if state.c > 1 and state.b == 1:
            return RiverState(state.m, state.c-2, 0)
     return None

# Take two cannibals from side 2 to 1
def c2_21(state):
     if 3-state.c > 1 and state.b == 0:
            return RiverState(state.m, state.c+2, 1)
     return None

# Take one missionary and one cannibal from side 1 to 2
def mc_12(state):
     if state.m > 1 and state.c > 1 and state.b == 1:
            return RiverState(state.m-1, state.c-1, 0)
     return None

# Take one missionary and one cannibal from side 2 to 1
def mc_21(state):
     if 3-state.m > 1 and 3-state.c > 1 and state.b == 0:
            return RiverState(state.m+1, state.c+1, 1)
     return None

In [3]:
def child_river_states(state):
    new_states = []
    if(m1_12(state)):
        new_states.append(m1_12(state))
    if(m1_21(state)):
        new_states.append(m1_21(state))
    if(m2_12(state)):
        new_states.append(m2_12(state))
    if(m2_21(state)):
        new_states.append(m2_21(state))
    if(c1_12(state)):
        new_states.append(c1_12(state))
    if(c1_21(state)):
        new_states.append(c1_21(state))
    if(c2_12(state)):
        new_states.append(c2_12(state))
    if(c2_21(state)):
        new_states.append(c2_21(state))
    if(mc_12(state)):
        new_states.append(mc_12(state))
    if(mc_21(state)):
        new_states.append(mc_21(state))
    return new_states